**Realizar las diferentes gráficas que permitan visualizar los resultados del modelo.**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

# Cargar los datos
data = pd.read_csv('Car details v3.csv')

# Renombrar las columnas para que coincidan con las nuevas variables
data = data.rename(columns={'name': 'Car_Name', 'year': 'Year', 'selling_price': 'Selling_Price',
                            'km_driven': 'Km_Driven', 'fuel': 'Fuel', 'seller_type': 'Seller_Type',
                            'transmission': 'Transmission', 'owner': 'Owner', 'mileage': 'Mileage',
                            'engine': 'Engine', 'max_power': 'Max_Power', 'torque': 'Torque',
                            'seats': 'Seats'})

# Identificar y manejar los valores faltantes
data_cleaned = data.dropna()

# Codificar las variables categóricas
data_encoded = pd.get_dummies(data_cleaned, columns=['Fuel', 'Seller_Type', 'Transmission'])

# Eliminar la columna 'Car_Name'
data_encoded.drop('Car_Name', axis=1, inplace=True)

# Escalar las características numéricas
scaler = StandardScaler()
numeric_features = ['Year', 'Selling_Price', 'Km_Driven', 'Owner']
data_encoded[numeric_features] = scaler.fit_transform(data_encoded[numeric_features])

# Dividir los datos en características (X) y etiquetas (y)
X = data_encoded.drop('Selling_Price', axis=1)
y = data_encoded['Selling_Price']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo con métricas de regresión
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error:', mse)
print('R^2 Score:', r2)

# Seleccionar las mejores características utilizando la prueba F de ANOVA
selector = SelectKBest(score_func=f_regression, k=5)  # Seleccionar las 5 mejores características
X_selected = selector.fit_transform(X, y)

# Obtener los índices de las características seleccionadas
selected_indices = selector.get_support(indices=True)

# Obtener los nombres de las características seleccionadas
selected_features = X.columns[selected_indices]

# Mostrar las características seleccionadas
print("Características seleccionadas:", selected_features)

# Gráfico de dispersión de las predicciones vs. los valores reales
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs. Valores reales')
plt.show()

# Gráfico de residuos
residuals = y_test - y_pred
plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, color='green')
plt.xlabel('Predicciones')
plt.ylabel('Residuos')
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residuos vs. Predicciones')
plt.show()